In [ ]:
import os
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import quantstats as qs
import copy
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import objective_functions
from datetime import datetime, timedelta
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt import CLA, plotting
from pypfopt import hierarchical_portfolio
from pypfopt import plotting
from metrics import *


DATA_DIR = (os.getcwd() + "/data/")
PREDICTION_DIR = (os.getcwd() + "/predictions_new/")
start_date = '2014-01-01'
end_date = "2019-11-30"
# Get fundamental data for each stock in the ticker and append to the dataframe


def get_all_symbols():
    return [v.strip('.csv') for v in os.listdir(DATA_DIR)]


def get_current_predictions():
    return [v.strip('_predictions.csv') for v in os.listdir(PREDICTION_DIR)]


tickers = get_all_symbols()
predicted_tickers = get_current_predictions()

sample_index = pd.read_csv(DATA_DIR + 'AAPL.csv',
                           index_col='Date', parse_dates=True)
mask = (sample_index.index >= start_date) & (sample_index.index <= end_date)
sample_index = sample_index.loc[mask]


In [1]:
def generate_hrp_models(start_month, start_year, end_month, end_year, d, n):

    curr_year, curr_month = start_year, start_month
    dfs = {
        "d": [],
        "dfs": []
    }
    for d in range(1, 4):
        df = {
            'sharpes': [],
            'expected_return': [],
            'annual_volatility': [],
            'num_stocks': [],
            'weights': []
        }
    # while not (curr_year > end_year or (curr_year == end_year and curr_month > end_month)):
        for i in range(25, 275, 25):
            tickers, pred_vector = get_top_n_tickers(curr_year, curr_month, i)
            close_data = get_close_prices(curr_year, curr_month, d, tickers)
            predicted_returns = generate_predicted_historical_returns(
                curr_year, curr_month, d, tickers)
            # print(close_data)
            # print(predicted_returns)
            # mu = predicted_returns.mean()
            mu = expected_returns.mean_historical_return(close_data)
            # print(mu)
            S = CovarianceShrinkage(close_data).ledoit_wolf()
            # print(S)\

            hrp = hierarchical_portfolio.HRPOpt(mu, S)
            # hrp.add_objective(objective_functions.L2_reg, gamma=0.1)
            raw_weights = hrp.optimize()
            cleaned_weights = hrp.clean_weights()
            hrp.portfolio_performance(verbose=False)

            ex_return = hrp.portfolio_performance()[0]
            df['expected_return'].append(ex_return)
            df['num_stocks'].append(i)
            df['annual_volatility'].append(hrp.portfolio_performance()[1])
            df['sharpes'].append(hrp.portfolio_performance()[2])
            df['weights'].append(cleaned_weights)
        # print(ef.portfolio_performance()[0]) # sharpe ratio
        dfs["d"].append(d)
        dfs['dfs'].append(df)

    # Plot Graph
    # plt.plot(df['num_stocks'], df['sharpes'])
    # print(dfs)
    return dfs
